In [1]:
import gc
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, Date, Numeric, Float, MetaData
from sqlalchemy.orm import sessionmaker

In [2]:
# データベースの接続情報を設定
SERVER_NAME = 'aaaaa\SQLEXPRESS'# SQLで「SELECT @@SERVERNAME」を実行する
DATABASE_NAME = 'C10000_database'
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'

In [3]:
def create_sqlalchemy_session(engine, schema_name,  table_name, columns):
    # テーブルのメタデータを作成
    metadata = MetaData(schema=schema_name)

    # テーブルのカラムを動的に定義
    my_table = Table(table_name, metadata, *columns)

    # セッションを作成
    Session = sessionmaker(bind=engine)
    session = Session()

    return metadata, my_table, session

In [4]:
engine = sqlalchemy.create_engine(f"mssql+pyodbc://{SERVER_NAME}/{DATABASE_NAME}?driver={DRIVER_NAME}")

In [5]:
SCHEMA_NAME= 'raw'
TABLE_NAME = 'testSQLAlchemy'

COLUMENS = [
    Column('決算日', Date),
    Column('勘定科目コード', String),
    Column('会社コード', String),
    Column('BS/PL/CF', String),
    Column('金額', Numeric),
]

In [6]:
# テーブル名とカラム定義を引数として渡して関数を呼び出す
metadata, table, session = create_sqlalchemy_session(engine, SCHEMA_NAME, TABLE_NAME, COLUMENS)

In [7]:
# データフレームを作成
data = {
    '決算日': ['2023-01-05', '2023-01-06'],
    '勘定科目コード': ['1003', '1004'],
    '会社コード': ['C', 'D'],
    'BS/PL/CF': ['BS', 'PL'],
    '金額': [10000.00, -5000.00]
}

df = pd.DataFrame(data)
df

,決算日,勘定科目コード,会社コード,BS/PL/CF,金額
0,2023-01-05,1003,C,BS,10000.0
1,2023-01-06,1004,D,PL,-5000.0


In [14]:
# データの挿入
date_l = df['決算日'].tolist()
accode_l = df['勘定科目コード'].tolist()
ccode_l = df['会社コード'].tolist()
kcode_l = df['BS/PL/CF'].tolist()
amt_l = df['金額'].tolist()

for date, acode, ccode, kcode, amt in zip(date_l, accode_l, ccode_l, kcode_l, amt_l):
    data = {'決算日': date, '勘定科目コード': acode, '会社コード': ccode, 'BS/PL/CF': kcode, '金額': amt}
    insert_stmt = table.insert().values(data)
    session.execute(insert_stmt)

# 変更をコミット
session.commit()

# セッションをクローズ
session.close()

In [15]:
gc.collect()

60